In [73]:
import matplotlib.pyplot as plt
import pandas as pd

from pearl.post_processing.bmi import calc_risk_by_group, clean_control


In [58]:
param_df = pd.read_parquet("/workspaces/pearlModel/out/S3_SA_10/combined/parameters.parquet").reset_index(drop=True)

In [59]:
param_df

,replication,group,new_dx,final_year,mortality_model,mortality_threshold_flag,idu_threshold,seed,bmi_intervention_scenario,bmi_intervention_start_year,...,sa_incidence_shift_smoking,sa_incidence_shift_anx,sa_incidence_shift_dpr,sa_incidence_shift_ckd,sa_incidence_shift_lipid,sa_incidence_shift_dm,sa_incidence_shift_ht,sa_incidence_shift_malig,sa_incidence_shift_esld,sa_incidence_shift_mi
0,0,het_black_female,base,2035,by_sex_race_risk,1,2x,130,3,2010,...,-0.037419,-0.000612,0.042113,-0.012015,-0.037355,0.049875,0.027092,0.023011,-0.046838,0.031034
1,1,het_black_female,base,2035,by_sex_race_risk,1,2x,131,3,2010,...,0.014130,0.019051,-0.037339,-0.026053,-0.024584,-0.007356,0.033900,0.024503,-0.011932,-0.023831
2,2,het_black_female,base,2035,by_sex_race_risk,1,2x,132,3,2010,...,0.028168,0.030160,-0.014310,0.046212,-0.014376,0.031858,0.040519,-0.045724,-0.048381,0.005605
3,3,het_black_female,base,2035,by_sex_race_risk,1,2x,133,3,2010,...,-0.035776,-0.006073,0.023790,-0.029744,-0.018884,0.005813,0.039605,-0.012805,0.043954,-0.018412
4,4,het_black_female,base,2035,by_sex_race_risk,1,2x,134,3,2010,...,0.038190,0.016507,0.038337,0.021167,0.016527,-0.037567,-0.046144,-0.029171,-0.023314,-0.035897
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,5,msm_white_male,base,2035,by_sex_race_risk,1,2x,5,3,2010,...,0.041861,-0.001159,0.011174,0.026591,0.001842,-0.020320,-0.031228,-0.041926,0.023844,-0.005869
146,6,msm_white_male,base,2035,by_sex_race_risk,1,2x,6,3,2010,...,-0.045830,-0.039234,0.009505,0.002982,-0.008119,-0.016459,0.012252,-0.006186,0.023588,0.001804
147,7,msm_white_male,base,2035,by_sex_race_risk,1,2x,7,3,2010,...,0.022347,0.047799,0.003850,0.000112,-0.042795,-0.023156,-0.000012,0.017923,0.030374,-0.011906
148,8,msm_white_male,base,2035,by_sex_race_risk,1,2x,8,3,2010,...,0.003086,-0.026727,-0.048860,-0.006953,-0.009765,0.002267,-0.002161,0.005536,0.004339,0.026090


In [60]:
bmi_int_dm_prev = pd.read_parquet("/workspaces/pearlModel/out/S3_SA_10/combined/bmi_int_dm_prev.parquet")

# clean to control specifications
control_bmi_int_dm_prev = clean_control(bmi_int_dm_prev, only_eligible=True)

# filter for only people eligible for intervention
bmi_int_eligible_risk = calc_risk_by_group(control_bmi_int_dm_prev, 7)

In [61]:
bmi_int_eligible_risk = bmi_int_eligible_risk.compute()

In [62]:
dm_incidence_params = param_df[['replication', 'group', 'sa_incidence_shift_dm']]

In [63]:
dm_incidence_params

,replication,group,sa_incidence_shift_dm
0,0,het_black_female,0.049875
1,1,het_black_female,-0.007356
2,2,het_black_female,0.031858
3,3,het_black_female,0.005813
4,4,het_black_female,-0.037567
...,...,...,...
145,5,msm_white_male,-0.020320
146,6,msm_white_male,-0.016459
147,7,msm_white_male,-0.023156
148,8,msm_white_male,0.002267


In [64]:
dm_incidence_sa = dm_incidence_params.merge(bmi_int_eligible_risk, on=['replication', 'group'], how='left').fillna(0)

In [65]:
dm_incidence_sa

,replication,group,sa_incidence_shift_dm,dm_num,num,risk
0,0,het_black_female,0.049875,6185.0,16254.0,0.380522
1,1,het_black_female,-0.007356,1098.0,14993.0,0.073234
2,2,het_black_female,0.031858,4845.0,16181.0,0.299425
3,3,het_black_female,0.005813,2672.0,17548.0,0.152268
4,4,het_black_female,-0.037567,48.0,17127.0,0.002803
...,...,...,...,...,...,...
145,5,msm_white_male,-0.020320,3.0,41543.0,0.000072
146,6,msm_white_male,-0.016459,9.0,44782.0,0.000201
147,7,msm_white_male,-0.023156,1.0,41979.0,0.000024
148,8,msm_white_male,0.002267,1695.0,42052.0,0.040307


In [66]:
dm_incidence_sa['positive_shift'] = dm_incidence_sa['sa_incidence_shift_dm'] > 0

In [69]:
dm_incidence_sa.groupby(['group', 'positive_shift']).quantile([0.05, 0.5, 0.95]).unstack()

replication            sa_incidence_shift_dm  \
                                       0.05 0.50  0.95                  0.05   
group            positive_shift                                                
het_black_female False                 1.45  4.5  8.40             -0.043668   
                 True                  0.50  4.5  7.75              0.010245   
het_black_male   False                 0.60  4.0  8.40             -0.039391   
                 True                  1.50  6.0  7.80              0.010286   
het_hisp_female  False                 0.30  2.5  6.40             -0.030696   
                 True                  1.75  5.5  8.75              0.012011   
het_hisp_male    False                 1.50  6.0  6.90             -0.039711   
                 True                  0.60  4.0  8.70              0.002226   
het_white_female False                 1.30  4.0  8.70             -0.038100   
                 True                  0.50  5.0  6.80              0.007932   
het_white_male   False                 0.30  4.5  7.85             -0.028832   
                 True                  1.50  4.5  8.25              0.004674   
idu_black_female False                 0.60  6.0  8.70             -0.047456   
                 True                  1.20  3.0  3.90              0.021670   
idu_black_male   False                 3.10  4.0  4.90             -0.046664   
                 True                  0.35  5.0  8.65              0.002332   
idu_hisp_female  False                 0.30  3.0  8.70             -0.040736   
                 True                  5.10  6.0  6.90              0.024683   
idu_hisp_male    False                 3.20  5.0  8.60             -0.038050   
                 True                  0.30  4.0  7.70              0.004246   
idu_white_female False                 0.10  1.0  1.90             -0.032264   
                 True                  3.30  6.0  8.70              0.001463   
idu_white_male   False                 1.45  5.5  7.85             -0.040747   
                 True                  0.50  4.0  8.25              0.009155   
msm_black_male   False                 1.00  6.0  7.80             -0.045468   
                 True                  1.20  3.0  8.00              0.005968   
msm_hisp_male    False                 1.10  2.0  3.80             -0.035507   
                 True                  0.90  6.0  8.70              0.007478   
msm_white_male   False                 1.35  4.5  8.30             -0.043713   
                 True                  0.40  4.0  7.60              0.004472   

                                                      dm_num           \
                                     0.50      0.95     0.05     0.50   
group            positive_shift                                         
het_black_female False          -0.033501 -0.010668    27.60     80.5   
                 True            0.033304  0.047980  3104.75   5105.0   
het_black_male   False          -0.019288 -0.002459    17.90    123.0   
                 True            0.022154  0.034202  1692.80   2321.0   
het_hisp_female  False          -0.020774 -0.002945    25.05    110.0   
                 True            0.028712  0.043748   806.75   1256.0   
het_hisp_male    False          -0.026132 -0.015058     2.00     11.0   
                 True            0.022106  0.045729   292.00    764.0   
het_white_female False          -0.021843 -0.003374    14.70     62.0   
                 True            0.009109  0.016261   649.00    685.0   
het_white_male   False          -0.020553 -0.007429     5.45     17.5   
                 True            0.035170  0.048654   223.50    638.5   
idu_black_female False          -0.042049 -0.004230     9.50     21.0   
                 True            0.027263  0.033810   459.00    630.0   
idu_black_male   False          -0.042752 -0.038840     1.30      4.0   
                 True            0.023789  0.041579   423.10    981.0   
idu_

In [74]:
# Figure setup
plt.figure(figsize=(6.5,8))

<Figure size 650x800 with 0 Axes>

<Figure size 650x800 with 0 Axes>